In [115]:
import os
import fastavro
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from correction.compute import *

In [121]:
data = pd.read_csv("../data_examples/csv/raw_detections.csv", index_col=False)

In [122]:
def correction(magnr, magpsf, sigmagnr, sigmapsf, isdiffpos): #Correction Algorithm 
    aux1 = 10**(-0.4 * magnr)
    aux2 = 10**(-0.4 * magpsf)
    aux3 = aux1 + isdiffpos * aux2
    if aux3 > 0:
        magpsf_corr = -2.5 * np.log10(aux3)
        aux4 = aux2**2 * sigmapsf**2 - aux1**2 * sigmagnr**2

        if aux4 >= 0:
            sigmapsf_corr = np.sqrt(aux4) / aux3
        else:
            sigmapsf_corr = ZERO_MAG
                
        sigmapsf_corr_ext = aux2 * sigmapsf / aux3
    else:
        magpsf_corr = ZERO_MAG
        sigmapsf_corr = ZERO_MAG
        sigmapsf_corr_ext = ZERO_MAG

    return magpsf_corr, sigmapsf_corr, sigmapsf_corr_ext

In [157]:
def apply_correction_df(data):
    df = data.copy()
    df['isdiffpos'] = df['isdiffpos'].map({'t': 1., 'f': -1.})
    correction_results = df.apply(lambda x: correction(x.magnr, x.magpsf, x.sigmagnr, x.sigmapsf, x.isdiffpos), axis=1, result_type="expand")
    df["magpsf_corr"], df["sigmapsf_corr"], df["sigmapsf_corr_ext"] = correction_results[0].values, correction_results[1].values, correction_results[2].values 
    return correction_results

In [160]:
%%timeit
a = apply_correction_df(data)

305 ms ± 23.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [163]:
grupos = data.groupby("objectId")

In [168]:
for g in grupos:
    re = apply_correction_df(g[1])